In [ ]:
import pandas as pd

ШАГ 1: ЗАГРУЗКА ДАННЫХ

In [ ]:
workers = pd.read_csv("workers.csv")
equipment = pd.read_csv("equipment.csv")
schedule = pd.read_csv("schedule_template.csv")
requirements = pd.read_csv("position_requirements.csv")
plan = pd.read_csv("plan.csv")

print("workers - workers.csv")
display(workers.head())
print("equipment - equipment.csv")
display(equipment.head())
print("schedule - schedule_template.csv")
display(schedule.head())
print("requirements - position_requirements.csv")
display(requirements.head())
print("plan - plan.csv")
display(plan.head())

In [ ]:
# Определяем основную профессию
cols = ["flat_printing", "letterpress_printing", "inkjet_printing"]
workers["основная_профессия"] = workers[cols].idxmax(axis=1)

# Добавляем все професии работника
workers["все_профессии"] = workers.apply(
    lambda row: [c for c in cols if row[c] > 0], axis=1
)

workers.head()

In [ ]:
# Параметры: генерируем для недели 1, понедельника, дневной смены
target_week = 2
# target_day = "Понедельник"
# target_shift = "day"
# shift = {"night": "day", "day": "evening", "evening": "night"}

In [ ]:
# Старая вспомогательная функция shift_next_week удалена.
# Используйте build_shift_rotation ниже для расчёта ротации смен.

In [ ]:
# Ранее здесь выполнялась проверка shift_next_week.
# Текущая версия рассчитывает кандидатов только через build_shift_rotation.

In [ ]:
def build_shift_rotation(
    schedule: pd.DataFrame, workers: pd.DataFrame, target_week: int
) -> pd.DataFrame:
    """
    Формирует общий датафрейм кандидатов на target_week для всех смен сразу,
    применяя правило переворота смен:
        night -> day, day -> evening, evening -> night
    """
    shift_map = {"night": "evening", "day": "night", "evening": "day"}

    # Базовый слой — прошлая неделя
    prev = schedule.loc[schedule["week"] == target_week - 1].copy()

    # Сохраним прошлую смену (на всякий случай для анализа)
    prev = prev.rename(columns={"shift": "prev_shift"})

    # Рассчитаем смену на следующую неделю и проставим неделю
    prev["shift"] = prev["prev_shift"].map(shift_map)
    prev["week"] = target_week

    # Объединим с данными по работникам
    result = prev.merge(workers, on="worker_id", how="left")

    # Короткая статистика по сменам
    counts = result["shift"].value_counts()
    total = int(counts.sum())
    print(f"Кандидаты на работу в неделю {target_week}: всего {total}")
    for s in ["day", "evening", "night"]:
        print(f"  {s}: {int(counts.get(s, 0))}")

    return result

In [ ]:
shift_workers_all = build_shift_rotation(schedule, workers, target_week)
display(shift_workers_all.head())

# # При необходимости — получить конкретную смену:
# shift_workers_day   = shift_workers_all.query('shift == "day"')
# shift_workers_even  = shift_workers_all.query('shift == "evening"')
# shift_workers_night = shift_workers_all.query('shift == "night"')

In [ ]:
# Для целевой смены получаем оборудование и требования по работникам
def f_shift_equipment(plan, shif_name):
    shift_equipment = plan[plan["shift"] == shif_name][
        ["week", "shift", "machine_id", "machine_type"]
    ].merge(requirements, on="machine_type", how="left")
    shift_equipment["worker_id"] = None

    return shift_equipment

In [ ]:
# Преобразуем в длинный формат
plan_long = plan.melt(
    id_vars="machine_id",
    value_vars=["night", "day", "evening"],
    var_name="shift",
    value_name="works",
)

# Оставляем только те строки, где машина работает
plan_long = (
    plan_long[plan_long["works"] == True].drop(columns="works").reset_index(drop=True)
)

# На удаление
# # Заменяем названия смен на русские
# plan_long = plan_long.replace(
#     {"shift": {"night": "night", "day": "day", "evening": "evening"}}
# )

plan_long = plan_long.merge(
    equipment[["machine_id", "machine_type"]], on="machine_id", how="left"
)
plan_long["week"] = target_week


# Создаем расписание для каждой смены с пустыми позициями работников
shift_equipment_day = f_shift_equipment(plan_long, "day")
shift_equipment_evening = f_shift_equipment(plan_long, "evening")
shift_equipment_night = f_shift_equipment(plan_long, "night")


display(shift_equipment_day.head())
display(shift_equipment_night.head())
display(shift_equipment_evening.head())

In [ ]:
def candidat(
    shift_workers_all,
    assigned_shift,
    global_assigned,
    mode,
    profession,
    min_rank,
    shift_name,
):
    assigned_shift = set(assigned_shift)
    global_assigned = set(global_assigned or set())
    blocked_ids = global_assigned - assigned_shift

    base_mask = (
        (shift_workers_all["shift"] == shift_name)
        & (~shift_workers_all["worker_id"].isin(blocked_ids))
        & (~shift_workers_all["worker_id"].isin(assigned_shift))
    )

    profession_mask = shift_workers_all["все_профессии"].apply(
        lambda profs: profession in profs
    )

    if mode == "ferst":
        primary_mask = shift_workers_all["основная_профессия"] == profession
        rank_mask = shift_workers_all[profession] == min_rank
        candidates = shift_workers_all[base_mask & primary_mask & rank_mask]
    elif mode == "second":
        rank_mask = shift_workers_all[profession].isin([min_rank, min_rank + 1])
        candidates = shift_workers_all[base_mask & profession_mask & rank_mask]
    elif mode == "third":
        rank_mask = shift_workers_all[profession] > 0
        candidates = shift_workers_all[base_mask & profession_mask & rank_mask]
    else:
        raise ValueError(
            f"Неизвестный режим '{mode}'. Используйте 'ferst', 'second' или 'third'."
        )

    return candidates.sort_values(by=[profession, "worker_id"], ascending=[False, True])


def position_assignment(
    shift_equipment,
    shift_workers_all,
    assigned_shift,
    global_assigned,
    mode="ferst",
    shift_name="day",
):
    free_positions = []
    updated = shift_equipment.copy()

    for index, row in updated.iterrows():
        worker_id = row.get("worker_id")
        if pd.isna(worker_id) or worker_id in ("", None):
            profession = row["machine_type"]
            min_rank = row["min_rank"]
            candidates = candidat(
                shift_workers_all,
                assigned_shift,
                global_assigned,
                mode,
                profession,
                min_rank,
                shift_name,
            )

            if not candidates.empty:
                chosen = candidates.iloc[0]
                updated.loc[index, "worker_id"] = chosen["worker_id"]
                assigned_shift.add(chosen["worker_id"])
            else:
                free_positions.append(updated.loc[index])

    columns = updated.columns
    free_df = (
        pd.DataFrame(free_positions, columns=columns)
        if free_positions
        else pd.DataFrame(columns=columns)
    )

    return free_df, updated, assigned_shift

In [ ]:
def run_assignment_for_shift(
    shift_equipment,
    shift_workers_all,
    default_rounds,
    global_assigned,
):
    assigned_shift = set()

    free_positions, updated, assigned_shift = position_assignment(
        shift_equipment,
        shift_workers_all,
        assigned_shift,
        global_assigned,
        mode=default_rounds[0][0],
        shift_name=default_rounds[0][1],
    )

    for round_idx, (mode, shift_name) in enumerate(default_rounds[1:], start=2):
        if free_positions.empty:
            break
        print(
            f"Остались свободные позиции после тура {round_idx - 1}: {len(free_positions)}"
        )
        free_positions, patch, assigned_shift = position_assignment(
            free_positions,
            shift_workers_all,
            assigned_shift,
            global_assigned,
            mode=mode,
            shift_name=shift_name,
        )
        updated = updated.combine_first(patch)

    return updated, assigned_shift

In [ ]:
def summary_team(shift_equipment):
    summary = shift_equipment.groupby(
        ["machine_id", "machine_type"], as_index=False
    ).agg(
        required=("position", "count"),
        assigned=("worker_id", lambda s: s.notna().sum() - (s == "").sum()),
    )
    return summary


def incomplete_team(shift_equipment):
    summary = summary_team(shift_equipment)
    incomplete = summary[
        (summary["assigned"] > 0) & (summary["assigned"] < summary["required"])
    ].copy()
    return incomplete


def decomlate_team(shift_equipment, assigned_shift):
    incomplete = incomplete_team(shift_equipment)
    destaff = incomplete[incomplete["required"] / 2 >= incomplete["assigned"]][
        "machine_id"
    ].to_list()

    if not destaff:
        return shift_equipment, assigned_shift

    mask = (
        shift_equipment["machine_id"].isin(destaff)
        & shift_equipment["worker_id"].notna()
    )
    freed = shift_equipment.loc[mask, "worker_id"].dropna().tolist()

    shift_equipment.loc[mask, "worker_id"] = None
    assigned_shift -= set(freed)

    print(f"Расформированы бригады: {destaff}")
    return shift_equipment, assigned_shift

In [ ]:
def staff_team(
    shift_equipment,
    shift_workers_all,
    assigned_shift,
    global_assigned,
    shift_name="night",
    mode="third",
):
    incomplete = incomplete_team(shift_equipment)
    if incomplete.empty:
        return shift_equipment, assigned_shift

    mask = shift_equipment["machine_id"].isin(incomplete["machine_id"])
    free_positions, patch, assigned_shift = position_assignment(
        shift_equipment.loc[mask].copy(),
        shift_workers_all,
        assigned_shift,
        global_assigned,
        mode=mode,
        shift_name=shift_name,
    )

    updated = shift_equipment.copy()
    if not patch.empty:
        updated.update(patch[["worker_id"]])

    return updated, assigned_shift

In [ ]:
# Назначение работников на позиции
default_tourse = [
    ("ferst", "day"),
    ("second", "day"),
    ("ferst", "night"),
    ("second", "night"),
    ("ferst", "evening"),
    ("second", "evening"),
]

default_tourse_day = default_tourse.copy()
default_tourse_evening = default_tourse[4:] + default_tourse[:4]
default_tourse_night = default_tourse[2:] + default_tourse[:2]

global_assigned = set()

shift_equipment_day, assigned_day = run_assignment_for_shift(
    shift_equipment_day,
    shift_workers_all,
    default_tourse_day,
    global_assigned,
)
shift_equipment_day, assigned_day = decomlate_team(shift_equipment_day, assigned_day)
shift_equipment_day, assigned_day = staff_team(
    shift_equipment_day,
    shift_workers_all,
    assigned_day,
    global_assigned,
    shift_name="day",
)
global_assigned.update(assigned_day)

shift_equipment_evening, assigned_evening = run_assignment_for_shift(
    shift_equipment_evening,
    shift_workers_all,
    default_tourse_evening,
    global_assigned,
)
shift_equipment_evening, assigned_evening = decomlate_team(
    shift_equipment_evening, assigned_evening
)
shift_equipment_evening, assigned_evening = staff_team(
    shift_equipment_evening,
    shift_workers_all,
    assigned_evening,
    global_assigned,
    shift_name="evening",
)
global_assigned.update(assigned_evening)

shift_equipment_night, assigned_night = run_assignment_for_shift(
    shift_equipment_night,
    shift_workers_all,
    default_tourse_night,
    global_assigned,
)
shift_equipment_night, assigned_night = decomlate_team(
    shift_equipment_night, assigned_night
)
shift_equipment_night, assigned_night = staff_team(
    shift_equipment_night,
    shift_workers_all,
    assigned_night,
    global_assigned,
    shift_name="night",
)
global_assigned.update(assigned_night)

In [ ]:
assigned_sets = {
    "day": assigned_day,
    "evening": assigned_evening,
    "night": assigned_night,
}

shift_tables = {
    "day": shift_equipment_day,
    "evening": shift_equipment_evening,
    "night": shift_equipment_night,
}

shift_gap_report = []
for shift_name, table in shift_tables.items():
    open_positions = int(table["worker_id"].isna().sum())
    shift_gap_report.append(
        {
            "shift": shift_name,
            "assigned_workers": len(assigned_sets[shift_name]),
            "open_positions": open_positions,
        }
    )
shift_gap_report = pd.DataFrame(shift_gap_report)
print(shift_gap_report.to_string(index=False))


def remaining_candidates(shift_workers_all, assigned_sets):
    rows = []
    for shift_name, assigned in assigned_sets.items():
        pool = shift_workers_all[shift_workers_all["shift"] == shift_name]
        free = pool[~pool["worker_id"].isin(assigned)]
        rows.append(
            {
                "shift": shift_name,
                "available_workers": int(len(free)),
            }
        )
    return pd.DataFrame(rows)


free_pool_report = remaining_candidates(shift_workers_all, assigned_sets)
print(free_pool_report.to_string(index=False))

unfilled_positions = pd.concat(
    [
        shift_equipment_day[shift_equipment_day["worker_id"].isna()],
        shift_equipment_evening[shift_equipment_evening["worker_id"].isna()],
        shift_equipment_night[shift_equipment_night["worker_id"].isna()],
    ],
    ignore_index=True,
)
print(f"Незаполненные позиции: {len(unfilled_positions)}")

In [ ]:
# Собираем все смены в один график night, day, evening
shift_equipment = pd.concat(
    [shift_equipment_night, shift_equipment_day, shift_equipment_evening],
    ignore_index=True,
)

In [ ]:
result = shift_equipment.merge(
    workers[["worker_id", "name"]],
    on="worker_id",
    how="left",
)

assigned_rows = result[result["worker_id"].notna()].copy()
assigned_rows = assigned_rows.sort_values(
    by=["shift", "machine_id", "position"],
    ignore_index=True,
)

assigned_rows.to_csv("assignment_output.csv", index=False, encoding="utf-8-sig")
print("assignment_output.csv обновлен")

schedule_wo_target = schedule[schedule["week"] != target_week].copy()
updated_schedule = pd.concat(
    [
        schedule_wo_target,
        assigned_rows[["week", "shift", "worker_id"]].drop_duplicates(),
    ],
    ignore_index=True,
)
updated_schedule.to_csv("schedule_template.csv", index=False, encoding="utf-8-sig")
print("schedule_template.csv обновлен")

In [ ]:
shift_equipment

In [ ]:
result

In [ ]:
assigned_rows[
            ["week", "shift", "machine_id", "position", "worker_id", "name"]
        ]

In [ ]:
brigade_summary = summary_team(shift_equipment)
print(brigade_summary.to_string(index=False))

In [ ]:
updated_schedule.head()

In [ ]:
assigned_rows[["week", "shift", "machine_id", "position", "worker_id", "name"]]